# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches': None}

# Imports
import configparser
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci

# Configure plotting in Jupyter
from matplotlib import pyplot as plt

mpl.rcParams['font.size'] = 7
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams['lines.markersize'] = 7

symbols = ['.','.','.','.']
colors = ['#0000ff','#999966','#dddd77', '#aaaaff']
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']
reproduce = [
    '2021-01-06 130248 bc84b67'
]

In [ ]:
def load_result_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    converge_time = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial, exp_converge = load_results(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
                converge_time[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
                if exp_converge is not None:
                    converge_time[s][n] = exp_converge[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance, converge_time

def load_result_run_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    converge_time = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial, exp_converge = load_result_runs(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
                converge_time[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
                if exp_converge is not None:
                    converge_time[s][n] = exp_converge[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance, converge_time

def load_results (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    score = np.load(experiment.get_filename('score.npy'))
    correct = np.load(experiment.get_filename('correct.npy'))
    distance_score = np.load(experiment.get_filename('distance_score.npy'))
    # Older code did not produce the below data
    try:
        initial_distance = np.load(experiment.get_filename('initial_distance.npy'))
    except FileNotFoundError:
        initial_distance = None
    try:
        converge_time = np.load(experiment.get_filename('converge_time.npy'))
    except FileNotFoundError:
        converge_time = None
    return experiment, score, correct, distance_score, initial_distance, converge_time

def load_result_runs (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    run_score = np.load(experiment.get_filename('score_run.npy'))
    run_correct = np.load(experiment.get_filename('correct_run.npy'))
    run_distance_score = np.load(experiment.get_filename('distance_score_run.npy'))
    # Older code did not produce the below data
    try:
        run_initial_distance = np.load(experiment.get_filename('initial_distance_run.npy'))
    except FileNotFoundError:
        run_initial_distance = None
    try:
        run_converge_time = np.load(experiment.get_filename('converge_time_run.npy'))
    except FileNotFoundError:
        run_converge_time = None
    return experiment, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time

def individual_title(config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
#        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
#            title_individual = 'Hill-Climb'
#        else:
#            title_individual = 'Mutation'
        title_individual = ''
        if individual_mode == slearn.MODE_ALL:
            title_individual += 'Serial-Indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += 'Fallback-Indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += 'Parallel-Indiv.'
    else:
        title_individual = 'social-only'
    return title_individual

def result_title (config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'hill-climb'
        else:
            title_individual = 'mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' serial indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' fallback indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' parallel indiv.'
    else:
        title_individual = 'social-only'
    title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
        config.getint('abm', 'N'),
        config.getint('abm', 'bit_count'),
        config.getint('abm', 'K'),
        config.getfloat('abm', 'nk_exponent', fallback=1),
        config.getint('strategies', 'sample', fallback=None),
        {True: 'critical', False: 'non-critical'}[config.getboolean('strategies', 'critical')],
        title_individual)
    return title

def nested_label(data, fig, ax, width, height=1, offset=(0, 0), fontsize=9):

    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    layer_width = width[0]
    cell_height = height / len(data)
    
    for i, data in enumerate(reversed(data)):
        rect = mpl.patches.Rectangle(
            (offset[0] + lw, offset[1] + i * cell_height), width=layer_width, height=cell_height,
            lw=1, edgecolor='black', facecolor="white",
            transform=label_trans, clip_on=False)
#        plt.text(
#            offset[0] + 0.02, offset[1] + i * cell_height + 0.04, data['label'],
#            fontsize=18,
#            transform=label_trans, rotation='vertical')
        plt.text(
            offset[0] + 0.02, offset[1] + (i + 0.5) * cell_height, data['label'],
            fontsize=fontsize, verticalalignment='center',
            transform=label_trans, rotation='vertical')
        ax.add_patch(rect)
        if 'data' in data:
            inner_offset = (
                offset[0] + layer_width,
                offset[1] + (i) * cell_height)
            nested_label(data['data'], fig, ax, width[1:], height=cell_height, offset=inner_offset)


In [ ]:
experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.figure(figsize=(6.66, 7.77))

for i, strat in enumerate(learning_strategies):
    if strat == 'Local Majority':
        continue
    plt.subplot(len(learning_strategies) - 1, 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        xlabels.append(net)
        y = score[strat][net][0]
        t, best = max(enumerate(y), key=lambda x: x[1])
        ys.append(best)
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=18)
    plt.errorbar(
        ys, range(len(xlabels)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.xlim([0, 1])
    plt.ylim([-0.5, 3.5])
    plt.grid(True)
    plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
    plt.xticks(fontsize=16)
#    if i == 0:
plt.xlabel('Mean Solution Quality')


plt.tight_layout()
out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.figure(figsize=(6.66,7.77))

for i, strat in enumerate(learning_strategies):
    if strat == 'Local Majority':
        continue
    plt.subplot(len(learning_strategies) - 1, 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        xlabels.append(net)
        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=18)
    plt.errorbar(
        ys, range(len(xlabels)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)
    plt.ylim([-0.5, 3.5])
    plt.xlim([1, 220])
    plt.xscale('log')
    plt.grid(True)
    plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
    plt.xticks(fontsize=16)
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

plt.xlabel('Convergence Time')

#plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=12)
plt.tight_layout()
out = experiment.get_filename('convergence.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('convergence.eps'), dpi=600)


In [ ]:
plt.figure(figsize=(10,7.5))
exclude = ['Random', 'StochasticBlock', 'Lattice']

for i, strat in enumerate(learning_strategies):
    plt.subplot(2, 2,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        y = score[strat][net][0]
        err95 = 1.96 * np.array(initial_distance[strat][net][1]) / math.sqrt(runs)
        plt.plot(range(len(y)), y, label=net, color=colors[j])
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=16)
    plt.grid(True)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(loc='lower right', fontsize=12)
    plt.xlim([0, 75])
    plt.ylim([0, 0.75])
    plt.ylabel('Mean dist. from any initial belief (bits)')
    plt.xlabel('Iteration')
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

plt.tight_layout()
out = experiment.get_filename('initial_distance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('initial_distance.eps'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(3.33, 3))

xlim = [0.4, 1]

ax = plt.gca()

data = [
]

xlabels = []

reproduce = [
    '2021-01-06 130248 bc84b67'
]


experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

row = 0

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        
        # Skip excluded networks
        if net in exclude:
            continue
        
        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)
        
        y = score[strat][net][0]
        t, best = max(enumerate(y), key=lambda x: x[1])
        ys.append(best)
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])

    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data = [{'label': plot_title}] + data

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=5,
        linewidth=1,
        label=strat)
    plt.axhline(row - 0.5, color='#000000', linewidth=1)
    row += 1 * len(ys)
    plt.xlim(xlim)

for i in range(6):
    plt.fill_between([0, 1], i * 4 - 0.5, i * 4 + 1.5, color="#e0e0ff")

plt.xlim(xlim)
plt.ylim([-0.5, len(xlabels) - 0.5])
plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
plt.xlabel('Solution Quality')

plt.subplots_adjust(left=0.33, right=0.95, top=0.90)
left, bottom = fig.transFigure.inverted().transform(
    ax.transData.transform_point((xlim[0],0)))
right, top = fig.transFigure.inverted().transform(
    ax.transData.transform_point((xlim[1],1)))    

right_margin = 0.23
lw = 0.005
width = left - right_margin - lw
nested_label(data, fig, ax, [width], fontsize=7)

# Draw separators for network labels
for i in range(3):
    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    h = 1 / 3
    
    rect = mpl.patches.Rectangle(
        (left - right_margin, i / 3), width=right_margin, height=h,
        lw=1, edgecolor='black', facecolor="none",
        transform=label_trans, clip_on=False)
    ax.add_patch(rect)

axt = ax.secondary_xaxis('top')
plt.grid(True, axis='x', which='both')

out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(3.33, 3))
ax = plt.gca()

data = []

xlabels = []

reproduce = [
    '2021-01-06 130248 bc84b67'
]


experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

row = 0

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue

    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):

        if net in exclude:
            continue

        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)

        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
        
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data = [{'label': plot_title}] + data

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=8,
        label=strat)    
    row += 1 * len(ys)
    plt.axhline(row - 0.5, color='#000000')

plt.subplots_adjust(left=0.455, right=0.95, top=0.95)
left, bottom = fig.transFigure.inverted().transform(
    ax.transData.transform_point((0,0)))
right, top = fig.transFigure.inverted().transform(
    ax.transData.transform_point((1,1)))    
left = 0.455

right_margin = 0.2675
lw = 0.01
width = left - right_margin - lw
nested_label(data, fig, ax, [width], fontsize=7)

for i in range(3):
    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    h = 1 / 3
    
    rect = mpl.patches.Rectangle(
        (left - right_margin, i / 3), width=right_margin, height=h,
        lw=1, edgecolor='black', facecolor="none",
        transform=label_trans, clip_on=False)
    ax.add_patch(rect)

for i in range(6):
    plt.fill_between([1, 200], i * 4 - 0.5, i * 4 + 1.5, color="#e0e0ff")

axt = ax.secondary_xaxis('top')
plt.xscale('log')
plt.grid(True, axis='x')
plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
plt.xlabel('Iterations to Converge')
plt.xlim([1, 200])
plt.ylim([-0.5, 11.5])

out = experiment.get_filename('convergence.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('convergence.eps'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(1.875, 3))
ax = plt.gca()

data = []

xlabels = []

reproduce = [
    '2021-01-06 130248 bc84b67'
]


experiment, learning_strategies, networks, score, correct, distance_score, initial_distance, converge_time = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

row = 0

for i, strat in enumerate(reversed(learning_strategies)):
    if strat == 'Local Majority':
        continue

    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):

        if net in exclude:
            continue

        # Add labels and update to newer terminology
        xlabel = net
        if xlabel == 'Random Group':
            xlabel = 'Random Pod'
        xlabels.append(xlabel)

        y, std = converge_time[strat][net]
        ys.append(y)
        err95 = 1.96 * np.array(std) / math.sqrt(runs)
        err95s.append(err95)
        
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title = plot_title.replace(' ', "\n")
    # We're going bottom to top, so insert on beginning of list
    data = [{'label': plot_title}] + data

    plt.errorbar(
        ys, range(row, row + len(ys)), xerr=err95s,
        fmt='{}'.format(symbols[i]),
        markerfacecolor='{}ff'.format(colors[0]), color=colors[0], markeredgecolor='{}ff'.format(colors[0]),
        markersize=0,
        capsize=5,
        label=strat)    
    row += 1 * len(ys)
    plt.axhline(row - 0.5, color='#000000', linewidth=1)

plt.subplots_adjust(left=0.05, right=0.95, top=0.9)
left, bottom = fig.transFigure.inverted().transform(
    ax.transData.transform_point((0,0)))
right, top = fig.transFigure.inverted().transform(
    ax.transData.transform_point((1,1)))    
left = 0.455

right_margin = 0
lw = 0.005
width = left - right_margin - lw
#nested_label(data, fig, ax, [width], fontsize=7)

for i in range(6):
    plt.fill_between([1, 200], i * 4 - 0.5, i * 4 + 1.5, color="#e0e0ff")

axt = ax.secondary_xaxis('top')
plt.xscale('log')
plt.grid(True, axis='x')
plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0)
plt.xlabel('Iterations to Converge')
plt.xlim([1, 200])
plt.ylim([-0.5, 11.5])
plt.yticks([], [])

out = experiment.get_filename('convergence.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('convergence.eps'), dpi=600)


In [ ]:
reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

labels = []

strat_abbrev = {
    'Best Neighbor': 'BN',
    'Confident Neighbor': 'CN',
    'Conform': 'C'
}

net_abbrev = {
    'Pref. Attach.': 'PA',
    'Small World': 'SW',
    'Long Path': 'LP',
    'Random Group': 'RP'
}

data = np.zeros((12, 12))
row = 0
for strat1 in learning_strategies:

    if strat1 == 'Local Majority':
        continue

    for net1 in networks:

        if net1 in exclude:
            continue

        print(strat1, net1)
        
        labels.append(strat_abbrev[strat1] + ': ' + net_abbrev[net1])
        
        col = 0
        for strat2 in learning_strategies:

            if strat2 == 'Local Majority':
                continue

            for net2 in networks:

                if net2 in exclude:
                    continue

                for r in range(runs):
                    y1 = max(run_score[strat1][net1][r])
                    y2 = max(run_score[strat2][net2][r])
                    
                    if y1 > y2:
                        data[row][col] += 1
                    elif y1 == y2:
                        data[row][col] += 0.5
                        
                # Increment column
                col += 1
                
        # Increment row
        row += 1

# Convert from total to fraction
data = data / runs

In [ ]:
plt.figure(figsize=(3, 3))

cdict = {
    'red': [(0, 0, 0),
            (0.495, 0.5, 0.33),
            (0.5, 0.33, 0.33),
            (0.505, 0.33, 0.5),
            (1, 1.0, 1.0)],
    'green': [(0, 0.0, 0.0),
              (0.495, 0.5, 0.33),
              (0.5, 0.33, 0.33),
              (0.505, 0.33, 0.0),
              (1, 1.0, 1.0)],
    'blue': [(0, 0.0, 0.0),
             (0.495, 0.0, 0.33),
             (0.5, 0.33, 0.33),
             (0.505, 0.33, 0.5),
             (1, 1, 1)],
}
cmp = LinearSegmentedColormap('netdelib', segmentdata=cdict, N=256)
im = plt.imshow(data, cmap=cmp)
plt.yticks(range(len(labels)), labels)
plt.xticks(range(len(labels)), labels, rotation=90)
for i in range(4, 12, 4):
    plt.axhline(i - 0.5, color="#000000", linewidth=3)
    plt.axvline(i - 0.5, color="#000000", linewidth=3)
    
for i in range(4, 12, 4):
    plt.axhline(i - 0.5, color="#ffffff", linewidth=1)
    plt.axvline(i - 0.5, color="#ffffff", linewidth=1)

plt.subplot()
plt.subplots_adjust(bottom=0.2, left=0.2, right=0.8)
cax = plt.axes([0.85, 0.24, 0.03, 0.6])
t = [0.0, 0.5, 1.0]
cbar = plt.colorbar(im, cax=cax, ticks=t)
cbar.ax.set_yticklabels(['0', '0.5', '1'])
plt.clim(0, 1)

plt.savefig(experiment.get_filename('result-frac-net-parallel.eps'))
plt.savefig(experiment.get_filename('result-frac-net-parallel.png'), dpi=600)


In [ ]:
reproduce = [
    '2021-02-04 161927 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

labels = []

strat_abbrev = {
    'Best Neighbor': 'BN',
    'Confident Neighbor': 'CN',
    'Conform': 'C'
}

net_abbrev = {
    'Pref. Attach.': 'PA',
    'Small World': 'SW',
    'Long Path': 'LP',
    'Random Group': 'RP'
}

data = np.zeros((12, 12))
row = 0
for strat1 in learning_strategies:

    if strat1 == 'Local Majority':
        continue

    for net1 in networks:

        if net1 in exclude:
            continue

        print(strat1, net1)
        
        labels.append(strat_abbrev[strat1] + ': ' + net_abbrev[net1])
        
        col = 0
        for strat2 in learning_strategies:

            if strat2 == 'Local Majority':
                continue

            for net2 in networks:

                if net2 in exclude:
                    continue

                for r in range(runs):
                    y1 = max(run_score[strat1][net1][r])
                    y2 = max(run_score[strat2][net2][r])
                    
                    if y1 > y2:
                        data[row][col] += 1
                    elif y1 == y2:
                        data[row][col] += 0.5
                        
                # Increment column
                col += 1
                
        # Increment row
        row += 1

# Convert from total to fraction
data = data / runs


In [ ]:
plt.figure(figsize=(3, 3))

cdict = {
    'red': [(0, 0, 0),
            (0.495, 0.5, 0.33),
            (0.5, 0.33, 0.33),
            (0.505, 0.33, 0.5),
            (1, 1.0, 1.0)],
    'green': [(0, 0.0, 0.0),
              (0.495, 0.5, 0.33),
              (0.5, 0.33, 0.33),
              (0.505, 0.33, 0.0),
              (1, 1.0, 1.0)],
    'blue': [(0, 0.0, 0.0),
             (0.495, 0.0, 0.33),
             (0.5, 0.33, 0.33),
             (0.505, 0.33, 0.5),
             (1, 1, 1)],
}
cmp = LinearSegmentedColormap('netdelib', segmentdata=cdict, N=256)
im = plt.imshow(data, cmap=cmp)
plt.yticks(range(len(labels)), labels)
plt.xticks(range(len(labels)), labels, rotation=90)
for i in range(4, 12, 4):
    plt.axhline(i - 0.5, color="#000000", linewidth=3)
    plt.axvline(i - 0.5, color="#000000", linewidth=3)
    
for i in range(4, 12, 4):
    plt.axhline(i - 0.5, color="#ffffff", linewidth=1)
    plt.axvline(i - 0.5, color="#ffffff", linewidth=1)

plt.subplot()
plt.subplots_adjust(bottom=0.2, left=0.2, right=0.8)
cax = plt.axes([0.85, 0.24, 0.03, 0.6])
t = [0.0, 0.5, 1.0]
cbar = plt.colorbar(im, cax=cax, ticks=t)
cbar.ax.set_yticklabels(['0', '0.5', '1'])
plt.clim(0, 1)

plt.savefig(experiment.get_filename('result-frac-net-fallback.eps'))
plt.savefig(experiment.get_filename('result-frac-net-fallback.png'), dpi=600)


In [ ]:
fig = plt.figure(figsize=(3, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

diff = [
    max(run_score['Best Neighbor']['Long Path'][r]) - max(run_score['Best Neighbor']['Small World'][r]) for r in range(1000)
]

diff_confident = [
    max(run_score['Confident Neighbor']['Random Group'][r]) - max(run_score['Confident Neighbor']['Pref. Attach.'][r]) for r in range(1000)
]

diff_conform = [
    max(run_score['Conform']['Random Group'][r]) - max(run_score['Conform']['Pref. Attach.'][r]) for r in range(1000)
]
plt.hist([diff_confident, diff_conform], bins=np.arange(-1.025,1.075,.05), label='Conform', zorder=2)
plt.grid()
plt.ylabel('Count')
plt.xlabel('Network Deliberation - Conventional')
plt.ylim([0, 500])
plt.xlim([-1.075, 1.075])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-netdelib-diff.png'), dpi=600)
plt.savefig(experiment.get_filename('results-netdelib-diff.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

xy = [
    (max(run_score['Best Neighbor']['Long Path'][r]),
     max(run_score['Best Neighbor']['Small World'][r]))
    for r in range(1000)
]
x, y = zip(*xy)
plt.plot(x, y, '.', markersize=1, color=colors[0])
plt.grid()
plt.title('Difference in Solution Quality')
locs, labels = plt.xticks()

xy = [
    (max(run_score['Confident Neighbor']['Long Path'][r]),
    max(run_score['Confident Neighbor']['Small World'][r]))
    for r in range(1000)
]
x, y = zip(*xy)
plt.plot(x, y, '.', markersize=1, color=colors[1])
plt.grid()
plt.ylabel('Count')
locs, labels = plt.xticks()


xy = [
    (max(run_score['Conform']['Random Group'][r]),
    max(run_score['Conform']['Pref. Attach.'][r]))
    for r in range(1000)
]
x, y = zip(*xy)
plt.plot(x, y, '.', markersize=1, color=colors[2])
plt.grid()
plt.ylabel('Count')
plt.xlabel('Network Deliberation - Conventional')

plt.tight_layout()
plt.savefig(experiment.get_filename('results-netdelib-diff.png'), dpi=600)
plt.savefig(experiment.get_filename('results-netdelib-diff.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 3))

plt.subplot(2, 1, 1)
plt.title('Conform: Quality Distribution')
plt.hist(
    [max(rs) for rs in run_score['Conform']['Pref. Attach.']],
    bins=np.arange(0,1.025,0.025), label="Preferential-Attachment",
    zorder=2)
plt.grid()
plt.ylabel('Count')
plt.ylim([0, 200])
locs, labels = plt.xticks()
plt.xticks(locs, labels=[])
plt.xlim([-0.025, 1.025])
plt.legend(fontsize=6,loc='upper left')

plt.subplot(2, 1, 2)
plt.hist(
    [max(rs) for rs in run_score['Conform']['Random Group']],
    bins=np.arange(0,1.025,0.025), label='Random-Pod',
    zorder=2)
plt.grid()
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0, 200])
plt.xlim([-0.025, 1.025])
plt.legend(fontsize=6,loc='upper left')

plt.tight_layout()
plt.savefig(experiment.get_filename('results-conform-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-conform-dist.eps'))


In [ ]:
fig = plt.figure(figsize=(3, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

diff = [
    max(run_score['Conform']['Random Group'][r]) - max(run_score['Conform']['Long Path'][r]) for r in range(1000)
]
plt.subplot(2,1,1)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label='Conform', zorder=2)
plt.grid()
plt.title('Difference in Solution Quality')
plt.ylabel('Count')
plt.ylim([0,450])
plt.legend(fontsize=6, loc="upper left")
locs, labels = plt.xticks()
plt.xticks(locs, labels=[])
plt.xlim([-1.075, 1.075])

diff = [
    max(run_score['Best Neighbor']['Random Group'][r]) - max(run_score['Best Neighbor']['Long Path'][r]) for r in range(1000)
]
plt.subplot(2,1,2)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label="Best-Neighbor", zorder=2)
plt.grid()
plt.ylabel('Count')
plt.xlabel('Random-Pod - Long-Path')
plt.ylim([0,450])
plt.xlim([-1.075, 1.075])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-netdelib-efficiency.png'), dpi=600)
plt.savefig(experiment.get_filename('results-netdelib-efficiency.eps'))

In [ ]:
fig = plt.figure(figsize=(6, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.subplot(2,2,1)
plt.hist(
    [max(rs) for rs in run_score['Conform']['Random Group']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conform: Random Group')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,3)
plt.hist(
    [max(rs) for rs in run_score['Conform']['Long Path']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conform: Long-Path')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Neighbor: Random Group')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Neighbor: Long Path')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.tight_layout()
plt.savefig(experiment.get_filename('results-netdelib-efficiency-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-netdelib-efficiency-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(6, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.subplot(2,2,1)
plt.hist(
    [max(rs) for rs in run_score['Confident Neighbor']['Pref. Attach.']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conf-Nbr: Pref-Attach')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Nbr: Pref-Attach')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,2)
plt.hist(
    [max(rs) for rs in run_score['Confident Neighbor']['Small World']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conf-Nbr: Small-World')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Nbr: Small-World')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.tight_layout()
plt.savefig(experiment.get_filename('results-confident-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-confident-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

diff = [
    max(run_score['Confident Neighbor']['Pref. Attach.'][r]) - max(run_score['Best Neighbor']['Pref. Attach.'][r]) for r in range(1000)
]
plt.subplot(2,1,1)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label="Pref-Attach", zorder=2)
plt.grid()
plt.title('Difference in Solution Quality')
plt.ylabel('Count')
plt.ylim([0,500])
plt.legend(fontsize=6, loc='upper left')
locs, labels = plt.xticks()
plt.xticks(locs, labels=[])
plt.xlim([-1.075, 1.075])

diff = [
    max(run_score['Confident Neighbor']['Small World'][r]) - max(run_score['Best Neighbor']['Small World'][r]) for r in range(1000)
]
plt.subplot(2,1,2)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label="Small-World", zorder=2)
plt.grid()
plt.ylabel('Count')
plt.xlabel('Confident-Neighbor - Best-Neighbor')
plt.ylim([0,500])
plt.xlim([-1.075, 1.075])
plt.legend(fontsize=6, loc='upper left')

plt.tight_layout()
plt.savefig(experiment.get_filename('results-confident.png'), dpi=600)
plt.savefig(experiment.get_filename('results-confident.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

ylim = 250
pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Best-Neighbor (Parallel)')
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    label="Pref-Attach", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    label="Small-World", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    label="Long-Path", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.yticks([0, 100, 200])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-bn-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-bn-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]

strat = 'Confident Neighbor'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Confident-Neighbor (Parallel)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-cn-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-cn-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]

strat = 'Conform'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Conform (Parallel)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-conf-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-conf-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Best-Neighbor (Fallback)')
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-bn-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-bn-fallback-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]

strat = 'Confident Neighbor'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Confident-Neighbor (Fallback)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")


plt.tight_layout()
plt.savefig(experiment.get_filename('results-cn-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-cn-fallback-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]

strat = 'Conform'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Conform (Fallback)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-conf-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-conf-fallback-dist.eps'))